In [8]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,Trainer

from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

In [15]:
nf8_config = BitsAndBytesConfig(
   load_in_8bit=True,
   bnb_8bit_quant_type="nf8",
   bnb_8bit_use_double_quant=True,
   bnb_8bit_compute_dtype=torch.bfloat16
)

config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.02,
        target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj","lm_head"],
        bias="none",
        task_type="CAUSAL_LM",
    )

In [16]:
model = AutoModelForCausalLM.from_pretrained("D:/Projects/project_yothalia/yothalia/server/model_weights/internlm/internlm-chat-7b-finetune", 
                                                quantization_config=nf8_config,
                                                #peft_config=config,
                                                trust_remote_code=True)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.89s/it]


In [17]:
model

InternLMForCausalLM(
  (model): InternLMModel(
    (embed_tokens): Embedding(103172, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x InternLMDecoderLayer(
        (self_attn): InternLMAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          (rotary_emb): InternLMDynamicNTKScalingRotaryEmbedding()
        )
        (mlp): InternLMMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): InternLMRMSNorm()
        (p

In [18]:
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 41,693,248 || all params: 7,363,674,176 || trainable%: 0.5662016950164418


In [19]:
model.save_pretrained("D:/Projects/project_yothalia/yothalia/server/model_weights/internlm/internlm-chat-7b-finetune-lora")

In [ ]:
prompt_template = """<s>[INST]<<SYS>>
{instruction}<</SYS>>
{history}
{user}[/INST]
{assisstant}</s>
"""